In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import re
import numpy as np
from os import listdir
from os.path import isfile, join
import pandas as pd
import time
import dask.dataframe as dd
import random
import spacy
from spacy import displacy
import gzip
import gensim 
import logging
from gensim.models.fasttext import FastText as FT_gensim
from gensim.test.utils import datapath
from spacy.matcher import PhraseMatcher
import json
import jsonlines
from spacy.strings import StringStore
from spacy.tokens import Span
from flair.data import TaggedCorpus
from flair.data_fetcher import NLPTaskDataFetcher, NLPTask
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings
from typing import List
from flair.embeddings import FlairEmbeddings
import torch
from flair.visual.training_curves import Plotter
from flair.data import Sentence
from flair.models import SequenceTagger
from flair.data import TaggedCorpus
from flair.data_fetcher import NLPTaskDataFetcher
from flair.trainers import ModelTrainer
from flair.training_utils import Metric
import nerviz as nv
import accuracy as ac
import matplotlib.pyplot as plt
from fastai.text import *
import warnings
warnings.filterwarnings('ignore')

C:\Users\Martin\Anaconda3\envs\deeplearning\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Import dataset

In [ ]:
# define columns
columns = {0: 'text', 1:'ner'}
data_folder = 'C:/Users/Martin/Desktop/Notebooks/speciale_ner/finaldataNER/'
corpus: TaggedCorpus = NLPTaskDataFetcher.load_column_corpus(data_folder, columns,
                                                              train_file='train.txt',
                                                              test_file='test.txt',
                                                              dev_file='val.txt')

In [ ]:
print(corpus.obtain_statistics())

## Load embeddings

In [ ]:
tagger: SequenceTagger = SequenceTagger.load_from_file('resources/taggers/example-1nertiw2/best-model.pt')

In [ ]:
sentences = [### SENTENCES should be here]

# # predict NER tags
# tagger.predict(sentence)
dis_predictions = []
for sen in sentences:   
    sss = Sentence(sen,use_tokenizer=True)
    aa = tagger.predict(sss)
    
    print(sss.to_tagged_string())
    print(sss.get_spans('ner'))
    dis_predictions.append((sss.to_tagged_string(),sss.get_spans('ner'),sss))
    print("***")



# print sentence with predicted tags
#print(sentence.to_tagged_string())

In [ ]:
# compare this to run 4
# 2019-06-04 01:14:46,229 MICRO_AVG: acc 0.8254 - f1-score 0.9043
# 2019-06-04 01:14:46,230 MACRO_AVG: acc 0.9116 - f1-score 0.9515

In [ ]:
tagger.eval()

In [ ]:
result_test = tagger.predict(corpus.test)

In [ ]:
# Search string:  atrial premature depolarizations 
# returned:  (0.6200701346711033, 'ectopic atrial beats')

# ['atrial pacemaker artifact ', 'pacemaker capture ']
# (['C0340914', 'C0340923'], [['234', '259'], ['241', '250', '268', '275']])
# Sinus rhythm and underlying sinus rhythm without ventricular sensing of the atrial pacemaker. There is intermittent appearance of apparent atrial pacemaker artifact without capture. Clinical correlation is suggested. TRACING #2  

# ['Sinus tachycardia ', 'chronic pulmonary disease ', 'sinus tachycardia ']
# (['C0039239', 'C0746102', 'C0039239'], [['113', '130'], ['255', '280'], ['325', '342']])
# Baseline artifact. Sinus tachycardia. Indeterminate QRS axis. Low limb leads voltage - is nonspecific. Prominent inferolateral Q waves - are nondiagnostic. Consider chronic pulmonary disease. Since previous tracing of [**2011-09-05**], sinus tachycardia present  

In [ ]:
counter = 0
summer = 0
for sent in corpus.test:
    k: Span = sent.get_spans('ner')
    for i in k:
        #print(len(i.tokens))
        summer +=len(i.tokens)
        counter += 1
        
print(summer/counter)

In [ ]:
trainer: ModelTrainer = ModelTrainer(tagger, corpus)

In [ ]:
??tagger

In [ ]:
predicted: Sentence = corpus.test[90]
vis2 = nv.NerVisualizer(predicted)
vis2.plt()

In [ ]:
vis2.html

In [ ]:
acc = ac.Accuracy(tagger, corpus.test)

In [ ]:
trues, preds = acc.get_preds_trues(mes_err = True,print_table=False)

In [ ]:
preds

In [ ]:
trues

In [ ]:
def dim(a):
    if not type(a) == list:
        return []
    return [len(a)] + dim(a[0])

In [ ]:
#acc.error_table

In [ ]:
dim(acc.error_table)

In [ ]:
import operator

def get_wrongs():
    sorted_x ={}
    wordza =  {}
    for doc in acc.error_table:
        for line in doc:
            if line[0] != line[1]:
               # print(line[0], line[1])
                if line[2] in wordza:
                    wordza[line[2]] = 1 +  wordza[line[2]]
                else:
                    wordza[line[2]] = 1


    return sorted(wordza.items(), key=lambda kv: kv[1],reverse=True)
    
        #print(line)
sortedx = get_wrongs()
def plot_n_wrongs(n: int ,sortedx: dict):

    topti = sortedx[:n]
    x = np.arange(n)
    x_names = list(map(lambda x: x[0],topti))
    yval = list(map(lambda x: x[1],topti))
    yval_norm = [i/sum(yval) for i in yval]
    plt.bar(x,yval_norm)
    ax = plt.xticks(x, x_names,rotation=45)
    plt.tight_layout()
    plt.savefig('barplotmestforkert.svg')
plot_n_wrongs(10,sortedx)

In [ ]:
acc.get_acc('POS_DIS')

In [ ]:
#trues, preds = acc.get_preds_trues(mes_err = False)
acc.get_acc('POS_DIS')
acc.plot_confusion_matrix(normalize=True,cmap=plt.cm.Blues)

In [ ]:
# #trues, preds = acc.get_preds_trues(mes_err = False)
# acc.get_acc('POSDIS')
# acc.plot_confusion_matrix(normalize=False,cmap=plt.cm.Blues)

dev score 3x cross validation  precision: 0.8889 - recall: 0.9182 

2019-05-13 19:28:29,834 POSDIS     tp: 696 - fp: 87 - fn: 62 - tn: 696 - precision: 0.8889 - recall: 0.9182 - accuracy: 0.8237 - f1-score: 0.9033

##  Negation Detektion Classifier

### Preprocessering

In [ ]:
# define columns
columns = {0: 'text', 1:'ner'}
data_folder = 'C:/Users/Martin/Desktop/Notebooks/speciale_ner/datenegreal/'
corpus_context: TaggedCorpus = NLPTaskDataFetcher.load_column_corpus(data_folder, columns,
                                                              train_file='train.txt',
                                                              test_file='test.txt',
                                                              dev_file='val.txt')

In [ ]:
import pdb;

In [ ]:
import pandas as pd
import re

def add_row(dataset, text, target,realword):
    #df2 = pd.DataFrame([[text, target]], columns = ['text','target'])
    #dataset['text','target'] =[text, target]
    dataset = dataset.append({'text':text, 'target':target, 'realword':realword},ignore_index=True)
    return dataset



def sentence_2_df(df,sentence):
    sent = sentence
   # print(len(sentence))
    spans_in_txt = []
    for span in sent.get_spans('ner'): 
        idx = [token.idx for token in span.tokens]    
        spans_in_txt.append((idx,span.tag))
        #print(span.tag)

    marker = ""
    #print(spans_in_txt)
   # return 
    #pdb.set_trace()
    index = len(sent.get_spans('ner'))
   # print(index) 
  #  print(spans_in_txt,sent.get_spans('ner'))
   # return None
    
    

    for idx in range(index):
        marker = ""
        for i in range(1,len(sent)+1): #ta tag et ord ad gangen

            if spans_in_txt[idx][0][0]==i:
                #print(spans_in_txt[idx][0][0],i)
                #break
                marker += "xxstart "
            elif spans_in_txt[idx][0][-1]==i-1:
                marker += "xxslut "
            val = sent.get_token(i)
            marker += val.text + ' ' 
            #print(val.text)
            
        #print(marker, "\n",spans_in_txt[idx][1])
        if spans_in_txt[idx][1] != "":
            #print(marker)
            
            #replaced = re.sub('(?<=xxstart)(.*)(?=xxslut)', 'xxdisease ', marker)
            rword = re.findall('(?<=xxstart)(.*)(?=xxslut)',marker)
            replaced = re.sub('xxstart(.*)xxslut', 'xxstart', marker)
            
            df = add_row(df, replaced, spans_in_txt[idx][1],rword)
    return df

In [ ]:
df = pd.DataFrame()
for test in corpus_context.train:
    df = sentence_2_df(df, test)

for test in corpus_context.test:
    df = sentence_2_df(df, test)
    
for test in corpus_context.dev:
    df = sentence_2_df(df, test)
data = df

In [ ]:
corpus_context

In [ ]:

df2 = pd.DataFrame()
for i in range(len(data)):
    #print(data.iloc[0].text, " \n")
    #p = "Good morning Dr. Adams. The patient is waiting for you in room number 3."
    mus = tokenize.sent_tokenize(data.iloc[i].text)
    for sent in mus:
        if 'xxstart' in sent:
            df2 = df2.append({'text':sent, 'target':data.iloc[i].target},ignore_index=True)
   #         print(sent)
    

In [ ]:
df2

In [ ]:
mus

In [ ]:
data.iloc[0].text

In [ ]:
df

In [ ]:
path = 'C:/Users/Martin/Desktop/Notebooks/speciale_ner'

bs = 16
data_lm = load_data(path, 'data_lm.pkl', bs=bs)

In [ ]:
df2.to_csv('fi.csv',index=False)

In [ ]:

data_for_clas = pd.read_csv('fi.csv') #xxstart <--- the disease


In [ ]:
df = data_for_clas

In [ ]:
#data_lm = load_data(path, 'data_lm.pkl', bs=bs)

In [ ]:
# 14 er den der er test
# 17 er validering

In [ ]:



data_clas = (TextList.from_df(data_for_clas, vocab=data_lm.vocab,cols='text')
             #grab all the text files in path
             .split_by_rand_pct(0.30,seed=17)
             #split by train and valid folder (that only keeps 'train' and 'test' so no need to filter)
             .label_from_df(classes=['POS_DIS', 'NEG_DIS','UN_DIS'],cols='target')
             #label them all with their folders
             .databunch(bs=bs))


In [ ]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5,pretrained=True) # metrics=[F1()]
learn.load_encoder('fine_tuned_enc_final')

In [ ]:
data_clas.train_ds[60]

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
#learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7))

In [ ]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7))

In [ ]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7))
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))

In [ ]:
# 0.91 procent uden prætræning overhovedet
# 0.93 med prætræning

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(1e-7/(2.6**4),1e-3), moms=(0.8,0.7))

In [ ]:
0	0.181281	0.208837	0.936073	00:56

In [ ]:
learn.load('sent_classifier') # 
#learn.save('fit_classifier') # 
#learn.save('sent_classifier') # 

In [ ]:
preds, targets = learn.get_preds(ds_type=DatasetType.Valid, ordered=True)

predictions = np.argmax(preds, axis = 1)
pd.crosstab(predictions, targets)

In [ ]:
preds, targets = learn.get_preds(ds_type=DatasetType.Valid, ordered=True)

predictions = np.argmax(preds, axis = 1)
pd.crosstab(targets,predictions)

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
pred_y = np.argmax(preds.numpy(),axis=1)
targets_y = targets.numpy()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

# import some data to play with
# iris = datasets.load_iris()
# X = iris.data
# y = iris.target
# class_names = iris.target_names

# # Split the data into a training set and a test set
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# # Run classifier, using a model that is too regularized (C too low) to see
# # the impact on the results
# classifier = svm.SVC(kernel='linear', C=0.01)
# y_pred = classifier.fit(X_train, y_train).predict(X_test)


def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    #classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normaliseret confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
   # cm *=100.0

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='Sand label',
           xlabel='Prædikteret label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    fig.set_size_inches(6, 6)
    fig.savefig('umlfittest2.svg')
    
    return ax


#np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
#plot_confusion_matrix(y_test, y_pred, classes=class_names,
                    #  title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
class_names =['Bekræftet','Negeret','Ubekræftet']
plot_confusion_matrix(targets_y, pred_y, classes=class_names, normalize=False,
                      title='Confusion matrix')

plt.show()

In [ ]:
pred_y = np.argmax(preds.numpy(),axis=1)
targets_y = targets.numpy()

In [ ]:

from sklearn.metrics import classification_report
target_names = ['POS_DIS', 'NEG_DIS', 'UN_DIS']
print(classification_report(targets_y, pred_y, target_names=target_names))

In [ ]:
f1_score(targets_y, pred_y, average='micro') 

In [ ]:

all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

In [ ]:
pred_y = np.argmax(preds.numpy(),axis=1)
targets_y = targets.numpy()

In [ ]:
from sklearn.preprocessing import label_binarize
y = label_binarize(targets_y, classes=[0, 1, 2])
prediz = preds.numpy()


In [ ]:
prediz.shape

In [ ]:
y.shape[1]

## AUC ROC

In [ ]:
# Learn to predict each class against the other

import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(3):
    fpr[i], tpr[i], _ = roc_curve(y[:, i], prediz[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y.ravel(), prediz.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
n_classes = y.shape[1]
# Plot linewidth.
lw = 2

In [ ]:
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure(1)
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Falsk positiv Rate')
plt.ylabel('Sand Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()


# Zoom in view of the upper left corner.
plt.figure(2)
plt.xlim(0, 0.2)
plt.ylim(0.6, 1.1)
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Zoom')
plt.legend(loc="lower right")
plt.show()

In [ ]:
pd.options.display.max_rows=1000
dataset = pd.DataFrame({'Preds':predictions.numpy(),'True':targets.numpy()})
dataset

## Error analysis

In [ ]:
learn.predict("normal sinus rhythm . prolonged qt interval baseline artifact since previous tracing baseline artifact present and xxstart has changed to cancer")

In [ ]:
learn.predict("sinus rhythm . short p - r interval of 0.10 seconds . qs deflections in leads vi - v2 . t wave inversions in leads i , ii , avl and v5-v6 which are non - specific . st segment elevation in leads vi - v2 shows that xxstart is present of undetermined age . compared to the previous tracing of _ date late ventricular premature beats were present immediately following p waves . anteroseptal myocardial infarction pattern of undetermined age was previously present . the p - r interval was normal rather than short . ")

In [ ]:
txt_ci = TextClassificationInterpretation.from_learner(learn)

In [ ]:
txt_ci.show_top_losses(k=100)

In [ ]:
^[ \t]*$\r?\n

In [ ]:
??txt_ci.show_intrinsic_attention

In [ ]:
learn.save('fit_classifier') # 

In [ ]:
??txt_ci

In [ ]:
data_lm.valid_ds

In [ ]:
txt_ci.html_intrinsic_attention(###INSERT ECG NOTE HER,cmap=cm.Purples)

In [ ]:
import matplotlib.cm as cm


def show_mistakes():
    wrongs = dataset.loc[dataset.Preds!=dataset["True"]]
    
    for i in wrongs.index:  
        print(learn.data.valid_ds[i][1], " Predicted: ", str(dataset.Preds[i]))
        txt_ci = TextClassificationInterpretation.from_learner(learn)
        test_text = learn.data.valid_ds[i][0]
        txt_ci.show_intrinsic_attention(test_text,cmap=cm.Purples)
        print("\n\n")
        
        #print(i)

In [ ]:
sinus rhythm with atrial premature beats . inferior wall myocardial infarction . consider apical - xxstart . consider posterior myocardial infarction . st - t wave abnormalities . since the previous tracing of _ date atrial premature beats are new . tracing 1

In [ ]:
show_mistakes()

In [ ]:
def prnt_str_iob(k=2):
    sds = ''
    for i in range(len(acc.error_table[k])): 
        #print(acc.error_table[k][i][2])
        sds += acc.error_table[k][i][2] + " "
    #print(sds)
    return sds, acc.error_table[k]

In [ ]:
acc.error_table

In [ ]:
#sss = Sentence("normal sinus rhytm . premature ventricular contractions . left ventricular hypertropy . inferior st elevation - myocardial injury is suspected . since last ecg , t wave changes laterally",use_tokenizer=True)
#sss = Sentence("sinus tachycardia . left atrial abnormality . left bundle - branch block . compared to the previous tracing of _date atrial arrhythmias are no longer recorded . otherwise , not change",use_tokenizer=True)
sss = Sentence("sinus rhytm with a - v conduction prolongation . left anterior fascicular block . probable old extensive anterior and inferior myocardial infarction with very low qrs voltage .",use_tokenizer=True)
aa = tagger.predict(sss)

In [ ]:
sentence = sss
a = 0
for i in range(1,len(sentence.tokens)):
    a += sentence.get_token(i).tags['ner'].score
    print(sentence.get_token(i), " " ,sentence.get_token(i).tags['ner'].value, " ", sentence.get_token(i).tags['ner'].score ) 


In [ ]:
??vis2.palette

In [ ]:
sss = Sentence(sss2,use_tokenizer=True)
aa = tagger.predict(sss)

In [ ]:
#predicted: Sentence = corpus.test[90]
#zoom 125
vis2 = nv.NerVisualizer(sss)
vis2.palette=['lightpink','green']
#vis2.plt2(sss2,[[5,7]])
vis2.plt()
#vis2.html

# norm

In [ ]:
from __future__ import print_function
from Authentication import *
import requests
import json
import argparse
#searchSCTresults=searchTerms.metode("sinus tachycardia")

In [ ]:
import searchTerms


In [ ]:
searchSCTresults=searchTerms.metode("sinus tachycardia")

In [ ]:
searchSCTresults

In [ ]:
sent: Sentence = dis_predictions[1][2]

In [ ]:
hej: Span = sent.get_spans('ner')[1]

In [ ]:
type(sent.get_spans('ner')[0])

Evaluation will be carried out according to the following F-scores:

Strict F-score: a predicted mention is considered a true positive if (i) its predicted span is exactly the same as for the gold-standard mention; and (ii) the predicted CUI is correct.  The predicted disorder is considered a false positive if the span is incorrect or the CUI is incorrect.

Relaxed F-score: a predicted mention is a true positive if (i) there is any word overlap between the predicted mention span and the gold-standard span (both in the case of contiguous and discontiguous spans); and (ii) the predicted CUI is correct. The predicted mention is a false positive if the span shares no words with the gold-standard span or the CUI is incorrect.

In [ ]:
def checksim(sentence: str, vocab: list, thres: float = 0.6):
    model = FT_gensim.load('xx.dd')
    if sentence == None: return
    candidates = []
    for term in vocab:
        try:
            cos_sim = model.wv.similarity(sentence,term)
        except:
            cos_sim = 0.0
            print("no ngrams !")
        
        if cos_sim>thres: candidates.append((cos_sim,term))
    
    sorted_terms = sorted(candidates, key=lambda tup: tup[0])
    if len(sorted_terms)>0: return sorted_terms[-1]
    else:                 return None
    
checksim('atrioventricular conduction delay',cand)
    

In [ ]:
model.wv.similarity('myocardial infarction', 'prior anteroseptal myocardial infarction')


In [ ]:
print(model.wv.similarity('prior anteroseptal myocardial infarction','myocardial infarction'))


In [ ]:
nlp.vocab.morphology.lemmatizer(u'endosomes', 'noun', morphology={'number': 'plur'})

In [ ]:
from srcalg.sieve import sieve
def sieve_algo(text):
    a = exact_macth(text)
    if a == 'NONE':
        a = hyphenation(text)
    if a == 'NONE':
        a = abreviation(text)
    if a == 'NONE':
        a = norepl(text)
    if a == 'NONE':
        a = partialmatch(text)
    return a

In [ ]:
abrlist = [## insert arblist here
          
          ]
number_converter = [('1st','first'),('2nd','second'),('3rd','third'),('4rd','fourth'),('5th','fifth'),('6th','sixth'),
                   ('7th','seventh'),('8th','eighth'),('9th','ninth'),('10th')]


In [ ]:
cand = ['old inferior myocardial infarction','old anterior myocardial infarction','anterior myocardial infarction','infarction','infarct','ectopic atrial beats','atrial fibrillation','sinus tachycardia','first degree atrioventricular block','atrioventricular conduction disorder','Ventricular ectopic beats','inferior myocardial infarction','acute anterolateral myocardial infarction','myocardial infarction']

In [ ]:
test_disorder = "1st degree a-v block"

In [ ]:
for i in dis_predictions:
    doc: Sentence = i[2]
   # print(span)
    print(doc)
    for span in doc.get_spans('ner'):
        searchSCTresults=searchTerms.metode(span.text)
        if searchSCTresults[0] == 'NONE':
            
            #print("Vector search ",checksim(span.text,cand))
            #print("vector match")
            searchSCTresults = checksim(span.text,cand) # partial match with wordvectors
            if searchSCTresults != None:
                searchSCTresults = searchSCTresults
            
            
        
     
        print("Search string: ",span.text, "\nreturned: " , searchSCTresults)
    print("*********************")
    print("\n\n\n\n\n")
    
    
    #print(i[1])
    #input("videre")

In [ ]:
fn = 1 +1 +1

In [ ]:
Search string:  atrial premature depolarizations 
returned:  (0.6200701346711033, 'ectopic atrial beats')
    


In [ ]:
Search string:  atrial premature depolarizations 
returned:  (0.6200701346711033, 'ectopic atrial beats')

['atrial pacemaker artifact ', 'pacemaker capture ']
(['C0340914', 'C0340923'], [['234', '259'], ['241', '250', '268', '275']])
Sinus rhythm and underlying sinus rhythm without ventricular sensing of the atrial pacemaker. There is intermittent appearance of apparent atrial pacemaker artifact without capture. Clinical correlation is suggested. TRACING #2  

['Sinus tachycardia ', 'chronic pulmonary disease ', 'sinus tachycardia ']
(['C0039239', 'C0746102', 'C0039239'], [['113', '130'], ['255', '280'], ['325', '342']])
Baseline artifact. Sinus tachycardia. Indeterminate QRS axis. Low limb leads voltage - is nonspecific. Prominent inferolateral Q waves - are nondiagnostic. Consider chronic pulmonary disease. Since previous tracing of [**2011-09-05**], sinus tachycardia present  

In [ ]:
junctional rhythm 

In [ ]:
def calculate_f1(tp,fn,fp)-> tuple:
    prec = tp/(tp+fp)
    recall = tp/(tp+fn)
    f = 2*(prec*recall/(prec+recall))
    return f, prec, recall

calculate_f1(104,3,2)

In [ ]:
searchSCTresults

In [ ]:
from gensim.models.fasttext import FastText as FT_gensim

In [ ]:
model = FT_gensim.load('xx.dd')

In [ ]:
corpus_file = datapath('dataset_lower_vec.cor')

#model_gensim = FT_gensim(size=50)
#model_gensim.build_vocab(corpus_file=corpus_file)
model_gensim.load('ftmodel.dd')

In [ ]:
similarities = model.wv.most_similar(positive=['1st'],topn=50)

In [ ]:
similarities

In [ ]:
model.wv.similarity('ischemia', 'sinus bradycardia')


In [ ]:
model.wv.similarity('prior anterior myocardial infarction','sinus bradycardia')

In [ ]:
print(model.wv.similarity('prior anterior myocardial infarction','old anterior myocardial infarction'))
print(model.wv.similarity('prior anterior myocardial infarction','anterior myocardial infarction'))

In [ ]:
model.wv.similarity('first degree atrioventricular block','1st degree a-v block')

In [ ]:
model.wv.similarity('vent. arrhyt.','sinus bradycardia')

In [ ]:
model.wv.similarity('sinus tachycardia','sinus bradycardia')

In [ ]:
model.wv.similarity('sin bradycardia','sinus bradycardia')

In [ ]:
model.wv.similarity('s. bra.','sinus bradycardia')

In [ ]:

model.wv.similarity('atrial ectopy','ectopic atrial beats')

In [ ]:

model.wv.similarity('atrial ectopy','atrial fibrillation')

In [ ]:
from fuzzywuzzy import process

In [ ]:
choices = ["prior anteroseptal myocardial infarction", "myocardial infarction"]
process.extract("sinus bradycardia", choices, limit=4)

In [ ]:
choices = ["vent. arrhyt.", "sinus tachycardia", "bradyarrhythmia", "sin bradycardia"]
process.extract("sinus bradycardia", choices, limit=4)

In [ ]:
print(model.wv.similarity('prior anteroseptal myocardial infarction','myocardial infarction'))
prior anteroseptal myocardial infarction

In [ ]:
from labella.scale import LinearScale
from labella.timeline import TimelineTex

In [ ]:
data = pd.read_csv(r'events.txt', parse_dates=True, index_col=0)
ax = GenerateTimeLine(data)
plt.show()

In [ ]:
import plotly.plotly as py
import plotly.figure_factory as ff

import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gantt_example.csv')

fig = ff.create_gantt(df, colors=['#333F44', '#93e4c1'], index_col='Complete', show_colorbar=True,
                      bar_width=0.2, showgrid_x=True, showgrid_y=True)
py.iplot(fig, filename='gantt-use-a-pandas-dataframe', world_readable=True)

In [ ]:
# baseline shizzle

In [ ]:
data_for_clas

In [ ]:
df = data_for_clas.copy()

In [ ]:
df

In [ ]:
https://etav.github.io/projects/spam_message_classifier_naive_bayes.html

In [ ]:
df['target'] = df.target.map({'POS_DIS':0, 'UN_DIS':1, 'NEG_DIS':2})
df.head()

In [ ]:
import string #punctuation
import pprint
from collections import Counter #frequencies

#Bag of Words from scratch
documents = ['Hello, how are you!',
             'Win money, win from home.',
             'Call me now.',
             'Hello, Call hello you tomorrow?']

lower_case_documents = []

for i in documents:
    lower_case_documents.append(i.lower())
print("lower case:", lower_case_documents)

# Remove punctuation.
sans_punctuation_documents = []

for i in lower_case_documents:
    sans_punctuation_documents = ["".join( j for j in i if j not in string.punctuation) for i in  lower_case_documents]
print("no punctuation:", (sans_punctuation_documents))

#Break each word
preprocessed_documents = []

for i in sans_punctuation_documents:
    preprocessed_documents.append(i.split(' ')) #split on space
print("break words:", (preprocessed_documents))

#Count frequency of words using counter
frequency_list = []

for i in preprocessed_documents:
    frequency_counts = Counter(i)
    frequency_list.append(frequency_counts)
print("tokenized counts:", pprint.pprint(frequency_list))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer() #set the variable

count_vector.fit(documents) #fit the function
count_vector.get_feature_names() #get the outputs

In [ ]:
doc_array = count_vector.transform(documents).toarray()
doc_array

In [ ]:
frequency_matrix = pd.DataFrame(doc_array,
                                columns = count_vector.get_feature_names()
                               )
frequency_matrix

## Creating baselines

In [ ]:
import pandas as pd
import nltk


df['tokenized_sents'] = df.apply(lambda row: ' '.join(i for i in nltk.word_tokenize(row['text'])), axis=1)
#df['tokenized_sents2'] = df.apply(lambda row: ' '.join(map(str, row)), axis = 1 )

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['tokenized_sents'],
                                                    df['target'],
                                                    test_size = 0.3)

print("Our original set contains", df.shape[0], "observations")
print("Our training set contains", X_train.shape[0], "observations")
print("Our testing set contains", X_test.shape[0], "observations")

In [ ]:
train = count_vector.fit_transform(X_train)
test = count_vector.transform(X_test)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

#https://www.kaggle.com/aneeshc/text-classification-using-bag-of-word-features

In [ ]:
https://www.kaggle.com/aneeshc/text-classification-using-bag-of-word-features

In [ ]:
from sklearn.svm import SVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
     ('clf', MultinomialNB())
    
    
])

# from sklearn.naive_bayes import MultinomialNB
# naive_bayes = MultinomialNB() #call the method
# naive_bayes.fit(train, y_train) #train the classifier on the training set
# predictions = naive_bayes.predict(X_test) #predic using the model on the testing set

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
predictions = pipeline.predict(X_test) #predic using the model on the testing set

In [ ]:
from sklearn.metrics import accuracy_score, precision_score,f1_score

print(accuracy_score(y_test,predictions))
print(f1_score(y_test,predictions,average=None))

In [ ]:
from sklearn.metrics import classification_report
target_names = ['POS_DIS', 'NEG_DIS', 'UN_DIS']
print(classification_report(y_test, predictions))

In [ ]:
from sklearn.metrics import classification_report
target_names = ['POS_DIS', 'NEG_DIS', 'UN_DIS']
print(classification_report(y_test, predictions))

In [ ]:
from sklearn.metrics import classification_report
target_names = ['POS_DIS', 'NEG_DIS', 'UN_DIS']
print(classification_report(y_test, predictions))

In [ ]:
from sklearn.metrics import classification_report
target_names = ['POS_DIS', 'NEG_DIS', 'UN_DIS']
print(classification_report(targets_y, pred_y, target_names=target_names))

In [ ]:
import gzip

for filename in ["ADMISSIONS.csv.gz"]:

    with gzip.open(filename, 'rt') as f:
         data = f.read()
    with open(filename[:-3], 'wt') as f:
         f.write(data)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import odsc_utils

df_adm_notes_clean = odsc_utils.load_clean_merge_dataset('ADMISSIONS.csv','NOTEEVENTS.csv')

In [ ]:
df_adm_notes_clean

In [4]:

df_adm_notes_clean['OUTPUT_LABEL'] = (df_adm_notes_clean.DAYS_NEXT_ADMIT < 30).astype('int')

In [ ]:
print('Number of positive samples:', (df_adm_notes_clean.OUTPUT_LABEL == 1).sum())
print('Number of negative samples:',  (df_adm_notes_clean.OUTPUT_LABEL == 0).sum())
print('Total samples:', len(df_adm_notes_clean))

In [5]:
# shuffle the samples
df_adm_notes_clean = df_adm_notes_clean.sample(n = len(df_adm_notes_clean), random_state = 42)
df_adm_notes_clean = df_adm_notes_clean.reset_index(drop = True)

# Save 30% of the data as validation and test data 
df_valid_test=df_adm_notes_clean.sample(frac=0.30,random_state=42)

df_test = df_valid_test.sample(frac = 0.5, random_state = 42)
df_valid = df_valid_test.drop(df_test.index)

# use the rest of the data as training data
df_train_all=df_adm_notes_clean.drop(df_valid_test.index)

In [ ]:
print('Test prevalence(n = %d):%.3f'%(len(df_test),df_test.OUTPUT_LABEL.sum()/ len(df_test)))
print('Valid prevalence(n = %d):%.3f'%(len(df_valid),df_valid.OUTPUT_LABEL.sum()/ len(df_valid)))
print('Train all prevalence(n = %d):%.3f'%(len(df_train_all), df_train_all.OUTPUT_LABEL.sum()/ len(df_train_all)))
print('all samples (n = %d)'%len(df_adm_notes_clean))
assert len(df_adm_notes_clean) == (len(df_test)+len(df_valid)+len(df_train_all)),'math didnt work'

print('Test samples ', len(df_test))

In [6]:
# split the training data into positive and negative
rows_pos = df_train_all.OUTPUT_LABEL == 1
df_train_pos = df_train_all.loc[rows_pos]
df_train_neg = df_train_all.loc[~rows_pos]

n = np.min([len(df_train_pos),len(df_train_neg)])

# merge the balanced data
df_train = pd.concat([df_train_pos.sample(n = n, random_state = 13), \
                      df_train_neg.sample(n = n, random_state = 13)],axis = 0)

# shuffle the order of training samples 
df_train = df_train.sample(n = len(df_train), random_state = 13).reset_index(drop = True)

print('Train prevalence (n = %d):'%len(df_train), df_train.OUTPUT_LABEL.sum()/ len(df_train))

Train prevalence (n = 4184): 0.5


In [ ]:
len(df_test_pos)

In [ ]:
len(df_test)

In [ ]:
df_test

In [ ]:
rows_pos = df_test.OUTPUT_LABEL == 1

In [ ]:
len(rows_pos)

In [ ]:
len(rows_pos)

In [ ]:
df_test

In [ ]:
df_train

In [7]:
def preprocess_text(df):
    # This function preprocesses the text by filling not a number and replacing new lines ('\n') and carriage returns ('\r')
    df.TEXT = df.TEXT.fillna(' ')
    df.TEXT =df.TEXT.str.replace('\n',' ')
    df.TEXT =df.TEXT.str.replace('\r',' ')
    return df

In [ ]:
df_train

In [8]:
# preprocess the text to deal with known issues
df_train = preprocess_text(df_train)
df_valid = preprocess_text(df_valid)
df_test = preprocess_text(df_test)

In [ ]:
df_train.TEXT = df_train["TEXT"].map(lambda x: x.lower())
df_test.TEXT = df_test["TEXT"].map(lambda x: x.lower())

In [ ]:
df_train

In [9]:

path = 'path'

bs = 16
data_lm = load_data(path, 'data_lm.pkl', bs=bs)




In [ ]:
df_train

In [ ]:
max_size = df_train['OUTPUT_LABEL'].value_counts().max()


lst = [df_train]
for class_index, group in df_train.groupby('OUTPUT_LABEL'):
    lst.append(group.sample(max_size-len(group), replace=True))
df_train = pd.concat(lst)


In [ ]:
data_clas.save()

In [11]:
data_clas = (TextList.from_df(df_train, cols='TEXT')
             #grab all the text files in path
             .split_by_rand_pct(0.15,seed=17)
             #split by train and valid folder (that only keeps 'train' and 'test' so no need to filter)
             .label_from_df(cols='OUTPUT_LABEL',classes=[0,1 ])
             #label them all with their folders
             .databunch(bs=bs))

In [ ]:
# https://github.com/mbsariyildiz/focal-loss.pytorch/blob/master/focalloss.py

In [ ]:
class FocalLoss(nn.Module):

    def __init__(self, gamma=2, alpha=1, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha, (float, int)): self.alpha = torch.Tensor([alpha, 1 - alpha])
        if isinstance(alpha, list): self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim()>2:
            input = input.view(input.size(0), input.size(1), -1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1, 2)                         # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1, input.size(2))    # N,H*W,C => N*H*W,C
        target = target.view(-1, 1)

        logpt = F.log_softmax(input, dim=1)
        logpt = logpt.gather(1,target)
        logpt = logpt.view(-1)
        pt = logpt.exp()

        if self.alpha is not None:
            if self.alpha.type() != input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0, target.data.view(-1))
            logpt = logpt * at

        loss = -1 * (1 - pt)**self.gamma * logpt
        if self.size_average: return loss.mean()
        else: return loss.sum()

In [ ]:
# From hackernoon
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduce = reduce
    def forward(self, inputs, targets):
        BCE_loss = loss(inputs, targets.squeeze(1))
       
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss
        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss

In [ ]:
learn.predict('this is ehh')

In [12]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.3,pretrained=False) # metrics=[F1()]
#learn.load_encoder('fine_tuned_enc_final')

In [ ]:
learn.load('level2')

In [ ]:
learn.loss_func = FocalLoss()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
https://forums.fast.ai/t/highly-imbalanced-data/37001/5

In [ ]:
torch.cuda.empty_cache()

In [13]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7))
# learn.freeze_to(-2)
# learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))
# learn.freeze_to(-3)
# learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))
# learn.unfreeze()
# learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.692435,0.782179,0.511962,17:50


In [ ]:
learn.save('level1')

In [ ]:
def pretty_size(size):
	"""Pretty prints a torch.Size object"""
	assert(isinstance(size, torch.Size))
	return " × ".join(map(str, size))

def dump_tensors(gpu_only=True):
	"""Prints a list of the Tensors being tracked by the garbage collector."""
	import gc
	total_size = 0
	for obj in gc.get_objects():
		try:
			if torch.is_tensor(obj):
				if not gpu_only or obj.is_cuda:
					print("%s:%s%s %s" % (type(obj).__name__, 
										  " GPU" if obj.is_cuda else "",
										  " pinned" if obj.is_pinned else "",
										  pretty_size(obj.size())))
					total_size += obj.numel()
			elif hasattr(obj, "data") and torch.is_tensor(obj.data):
				if not gpu_only or obj.is_cuda:
					print("%s → %s:%s%s%s%s %s" % (type(obj).__name__, 
												   type(obj.data).__name__, 
												   " GPU" if obj.is_cuda else "",
												   " pinned" if obj.data.is_pinned else "",
												   " grad" if obj.requires_grad else "", 
												   " volatile" if obj.volatile else "",
												   pretty_size(obj.data.size())))
					total_size += obj.data.numel()
		except Exception as e:
			pass        
	print("Total size:", total_size)

In [ ]:
dump_tensors()

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-3), moms=(0.8,0.7))


In [ ]:
learn.save('level2')

In [ ]:
learn.load('level2')

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))

In [ ]:
learn.save('level3')

In [ ]:
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))

In [14]:
preds, targets = learn.get_preds(ds_type=DatasetType.Valid, ordered=True)
pred_y = np.argmax(preds.numpy(),axis=1)
targets_y = targets.numpy()



In [ ]:
for i in preds:
    print(i)

In [ ]:
for i in targets:
    print(i)

In [ ]:
target_names = [0,1]
#print(classification_report(targets_y, pred_y, target_names=target_names))
f1_score(targets_y, pred_y, average='macro') 

In [ ]:
learn

In [15]:

predictions = np.argmax(preds, axis = 1)
pd.crosstab(targets_y,pred_y)

col_0,0,1
row_0,,
0,19,298
1,8,302


In [ ]:
from sklearn.metrics import f1_score
f1_score(targets, predictions.numpy(), average='weighted')


In [ ]:
len(preds[:,1].numpy())

In [16]:
from sklearn.metrics import roc_auc_score
auc_train = roc_auc_score(targets, preds[:,1])
#auc_valid = roc_auc_score(y_valid, y_valid_preds)

In [17]:
auc_train

0.566805739289712

In [ ]:
from sklearn import metrics

fpr, tpr, thresholds = metrics.roc_curve(targets, preds[:,1].numpy(), pos_label=1)
metrics.auc(fpr, tpr)


In [ ]:
tpr

In [ ]:
np.array([0, 0, 1, 1])

In [ ]:
preds[:,1].numpy()

In [ ]:
preds